<a href="https://colab.research.google.com/github/Suraj28295/glowing-waffle/blob/main/Create_streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run streamlit app from a Google Colab Notebook
> Created by [Manuel Romero](https://twitter.com/mrm8488)

In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 7.1MB 2.8MB/s 
     |████████████████████████████████| 4.4MB 45.5MB/s 
     |████████████████████████████████| 522kB 50.5MB/s 
     |████████████████████████████████| 112kB 51.3MB/s 
     |████████████████████████████████| 102kB 10.5MB/s 
     |████████████████████████████████| 122kB 48.4MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.3.0 which is incompatible.
ERROR: google-auth 1.7.2 has requirement cachetools<3.2,>=2.0.0, but you'll have cachetools 4.1.0 which is incompatible.


## Create a streamlit app example


## Install localtunnel

In [1]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- APP TITLE ---
st.title("🏖️ Retirement Planner & Income‑from‑Returns Simulator")

# --- 1. PERSONAL DETAILS & TIMELINE ---
st.sidebar.header("1. Personal & Timeline")
current_age      = st.sidebar.number_input("Current age", min_value=18, max_value=80, value=30)
retire_age       = st.sidebar.number_input("Retirement age", min_value=current_age+1, max_value=100, value=60)
life_expectancy  = st.sidebar.number_input("Life expectancy age", min_value=retire_age+1, max_value=120, value=85)
years_to_retire  = retire_age - current_age
post_retire_years= life_expectancy - retire_age

# --- 2. INFLATION & RETURN ASSUMPTIONS ---
st.sidebar.header("2. Inflation & Returns")
inflation        = st.sidebar.slider("Assumed inflation (%)", 0.0, 12.0, 6.0, 0.1)
# Asset buckets (same as before)
col1, col2, col3, col4 = st.sidebar.columns(4)
with col1:
    alloc_fd  = st.number_input("FD allocation (%)",  0, 100, 30)
    rate_fd   = st.number_input("FD gross rate (%)",  7.5)
with col2:
    alloc_p2p = st.number_input("P2P allocation (%)", 0, 100, 30)
    rate_p2p  = st.number_input("P2P gross rate (%)",12.0)
with col3:
    alloc_tf  = st.number_input("Tax‑Free Bond allocation (%)", 0, 100, 20)
    rate_tf   = st.number_input("Tax‑Free Bond gross rate (%)", 5.5)
with col4:
    alloc_eq  = 100 - alloc_fd - alloc_p2p - alloc_tf
    st.markdown(f"**Equity/MF allocation:** {alloc_eq}%")
    rate_eq   = st.number_input("Equity/MF gross rate (%)", 8.0)

# --- 3. TAX & COSTS ---
st.sidebar.header("3. Tax & Costs")
tax_slab      = st.sidebar.slider("Marginal tax rate (%)", 0, 40, 30)
ltdcg_rate    = st.sidebar.slider("LTCG tax rate (%)", 0.0, 20.0, 12.5)
exp_ratio_eq  = st.sidebar.slider("Eq/MF expense ratio (%)", 0.0, 3.0, 1.0)
exit_load     = st.sidebar.slider("Exit load (%)", 0.0, 2.0, 0.0)

# --- 4. CONTRIBUTION SETTINGS ---
st.sidebar.header("4. Contributions")
monthly_contrib= st.sidebar.number_input("Monthly contribution (₹)", value=20_000, step=1_000)
contrib_growth = st.sidebar.slider("Contribution growth (%) per year", 0.0, 15.0, 5.0, 0.1)

# --- 5. WITHDRAWAL SETTINGS ---
st.sidebar.header("5. Retirement Withdrawals")
target_income  = st.sidebar.number_input("Desired monthly income (today's ₹)", value=100_000, step=5_000)
swr            = st.sidebar.slider("Safe Withdrawal Rate (%)", 1.0, 6.0, 4.0)
buffer_months  = st.sidebar.slider("Liquidity buffer (months)", 0, 24, 6)

# --- 6. NET RATES & WEIGHTS ---
net_fd_rate   = rate_fd  * (1 - tax_slab/100)
net_p2p_rate  = rate_p2p * (1 - tax_slab/100)
net_tf_rate   = rate_tf   # tax‑exempt
gross_eq_adj  = rate_eq  - exp_ratio_eq
net_eq_rate   = gross_eq_adj * (1 - ltdcg_rate/100)

weights       = np.array([alloc_fd, alloc_p2p, alloc_tf, alloc_eq]) / 100
net_rates     = np.array([net_fd_rate, net_p2p_rate, net_tf_rate, net_eq_rate])
wtd_yield     = (weights * net_rates).sum()

# --- 7. INFLATION‑INDEX TARGET AT RETIREMENT ---
# Inflate today's income to retirement year
target_annual_today = target_income * 12
target_annual_ret   = target_annual_today * (1 + inflation/100) ** years_to_retire

# --- 8. CORPUS NEEDED AT RETIREMENT ---
corpus_yield  = target_annual_ret / (wtd_yield/100)
corpus_swr    = target_annual_ret / (swr/100)
buffer_amount = target_income * buffer_months

# --- 9. ACCUMULATION PHASE SIMULATION ---
# Year-by-year corpus build-up
years = list(range(0, years_to_retire+1))
corpus = [0.0]
for yr in years[1:]:
    prev = corpus[-1]
    # increase contribution
    contrib = monthly_contrib * 12 * (1 + contrib_growth/100) ** (yr-1)
    # growth
    new = (prev + contrib) * (1 + wtd_yield/100)
    corpus.append(new)

# --- 10. DISPLAY RESULTS ---
st.header("📊 Key Outcomes")
colA, colB = st.columns(2)
colA.metric("Weighted Net Yield",      f"{wtd_yield:.2f}%")
colA.metric("Inflated Annual Need @ Retire", f"₹ {target_annual_ret:,.0f}")
colB.metric("Corpus Needed (Yield‑based)",  f"₹ {corpus_yield:,.0f}")
colB.metric("Corpus Needed (SWR‑based)",    f"₹ {corpus_swr:,.0f}")
st.metric("Liquidity Buffer", f"₹ {buffer_amount:,.0f}")

# Accumulation chart
st.subheader("Accumulation Phase: Corpus Over Time")
fig1, ax1 = plt.subplots()
ax1.plot([current_age + y for y in years], np.array(corpus)/1e7)
ax1.set_xlabel("Age"); ax1.set_ylabel("Corpus (₹ crore)")
st.pyplot(fig1)

# --- 11. POST‑RETIREMENT SIMULATION ---
st.subheader("Post‑Retirement Distribution Simulation")
port = corpus[-1] - buffer_amount
ages = []
balances = []
for y in range(post_retire_years+1):
    age = retire_age + y
    # withdraw inflation‑indexed amount each year
    withdraw = target_annual_today * (1 + inflation/100) ** (years_to_retire + y)
    port = port * (1 + wtd_yield/100) - withdraw
    balances.append(port)
    ages.append(age)
    if port <= 0:
        break

fig2, ax2 = plt.subplots()
ax2.plot(ages, np.array(balances)/1e7)
ax2.set_xlabel("Age"); ax2.set_ylabel("Remaining Corpus (₹ crore)")
st.pyplot(fig2)

# Show table of first few years
df_post = pd.DataFrame({
    "Age": ages,
    "Balance (₹)": balances,
    "Balance (₹ lakh)": [b/1e5 for b in balances]
})
st.write(df_post.style.format({"Balance (₹)": "₹{:,.0f}", "Balance (₹ lakh)": "{:,.2f}"}))

# --- 12. SENSITIVITY ANALYSIS ---
st.subheader("Sensitivity: Corpus Needed vs SWR & Inflation")
infl_range = np.linspace(max(0, inflation-3), inflation+3, 30)
swr_range  = np.linspace(2, 6, 30)
# Simple 2D table (heatmap-style DataFrame)
sens_data = []
for i in infl_range:
    row = []
    for s in swr_range:
        req = target_annual_today * (1 + i/100)**years_to_retire / (s/100)
        row.append(req/1e7)
    sens_data.append(row)
sens_df = pd.DataFrame(
    sens_data,
    index=[f"{i:.1f}%" for i in infl_range],
    columns=[f"{s:.1f}%" for s in swr_range]
)
st.write("Corpus needed (₹ crore) for different Inflation vs SWR:")
st.dataframe(sens_df)

# --- 13. TAX‑SAVING & OTHER NOTES ---
st.subheader("Tax‑Saving & Notes")
elss = st.number_input("80C: ELSS investment (₹)", value=150_000, step=10_000)
st.write(f"You can claim ₹{elss:,.0f} under Section 80C.")
st.markdown("""
- You may adjust allocations over time (de-risk near retirement).
- Consider buffer in liquid assets for the first 1–2 years post‑retirement.
""")

Overwriting app.py


In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.261s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [6]:
%%writefile app.py

import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

st.title("Advanced Retirement & Income Planner")

# --- 1. USER TARGET & MACRO ASSUMPTIONS ---
st.sidebar.header("1. Income & Macro")
target_monthly = st.sidebar.number_input("₹ Desired monthly income", value=230_000, step=10_000, format="%d")
annual_need    = target_monthly * 12
inflation      = st.sidebar.slider("Assumed inflation (%)", 0.0, 12.0, 6.0)

# --- 2. TAX PARAMETERS ---
st.sidebar.header("2. Tax Settings")
tax_slab        = st.sidebar.slider("Marginal tax rate (%)", 0, 40, 30)
ltdcg_rate      = st.sidebar.slider("LTCG tax rate (%)", 0.0, 20.0, 12.5)
ltdcg_exempt    = st.sidebar.number_input("LTCG exemption (₹)", value=125_000, step=25_000)
tds_fd_threshold  = st.sidebar.number_input("FD TDS threshold (₹)", value=50_000, step=10_000)
tds_div_threshold = st.sidebar.number_input("Dividend TDS threshold (₹)", value=10_000, step=5_000)

# --- 3. ALLOCATIONS & GROSS RATES ---
st.sidebar.header("3. Asset Buckets")
col1, col2, col3, col4 = st.sidebar.columns(4)

with col1:
    alloc_fd  = st.number_input("FD allocation (%)",  0, 100, 30)
    rate_fd   = st.number_input("FD gross rate (%)",  7.5)
with col2:
    alloc_p2p = st.number_input("P2P allocation (%)", 0, 100, 30)
    rate_p2p  = st.number_input("P2P gross rate (%)",12.0)
with col3:
    alloc_tf  = st.number_input("Tax‑Free Bond allocation (%)", 0, 100, 20)
    rate_tf   = st.number_input("Tax‑Free Bond gross rate (%)", 5.5,
                                help="Typical AAA PSU yields ~5.4–5.5%")
with col4:
    alloc_eq  = 100 - alloc_fd - alloc_p2p - alloc_tf
    st.markdown(f"**Equity/MF allocation:** {alloc_eq}%")
    rate_eq   = st.number_input("Equity/MF gross rate (%)", 8.0)

# --- 4. FUND COSTS ---
st.sidebar.header("4. Fund Costs")
exp_ratio_eq = st.sidebar.slider("Eq/MF expense ratio (%)", 0.0, 3.0, 1.0)
exit_load    = st.sidebar.slider("Exit Load (%)", 0.0, 2.0, 0.0)

# --- 4A. CURRENT INVESTMENTS ---
st.sidebar.header("4A. Current Investments")
curr_eq = st.sidebar.number_input("Current Equity/MF corpus (₹)", value=500_000, step=10_000)
curr_fd = st.sidebar.number_input("Current FD corpus (₹)", value=300_000, step=10_000)
curr_p2p = st.sidebar.number_input("Current P2P corpus (₹)", value=0, step=10_000)
curr_tf = st.sidebar.number_input("Current Tax-Free Bond corpus (₹)", value=0, step=10_000)

# --- 5. SWR & Liquidity ---
st.sidebar.header("5. SWR & Liquidity")
swr           = st.sidebar.slider("Safe Withdrawal Rate (%)", 1.0, 6.0, 4.0)
buffer_months = st.sidebar.slider("Liquidity buffer (months)", 0, 12, 6)

# --- 6. Net Yields & Portfolio Metrics ---
net_fd_rate   = rate_fd    * (1 - tax_slab/100)
net_p2p_rate  = rate_p2p   * (1 - tax_slab/100)
net_tf_rate   = rate_tf     # fully tax‑exempt
gross_eq_adj  = rate_eq    - exp_ratio_eq
net_eq_rate   = gross_eq_adj * (1 - ltdcg_rate/100)

weights    = np.array([alloc_fd, alloc_p2p, alloc_tf, alloc_eq]) / 100
net_rates  = np.array([net_fd_rate, net_p2p_rate, net_tf_rate, net_eq_rate])
wtd_yield  = (weights * net_rates).sum()

corpus_yield = annual_need / (wtd_yield/100)
corpus_swr   = annual_need / (swr/100)
buffer_amount = target_monthly * buffer_months

# --- 7. DISPLAY RESULTS ---
st.header("Key Outcomes")
st.metric("Weighted Nominal Yield",      f"{wtd_yield:.2f}%")
st.metric("Corpus (Yield‑based)",       f"₹ {corpus_yield:,.0f}")
st.metric("Corpus (SWR‑based)",         f"₹ {corpus_swr:,.0f}")
st.metric("Liquidity Buffer",           f"₹ {buffer_amount:,.0f}")

# Detailed table
assets = ["FD", "P2P Lending", "Tax‑Free Bond", "Equity/MF"]
df = pd.DataFrame({
    "Allocation (%)": [alloc_fd, alloc_p2p, alloc_tf, alloc_eq],
    "Gross Rate (%)": [rate_fd, rate_p2p, rate_tf, rate_eq],
    "Net Rate (%)":   [round(net_fd_rate,2), round(net_p2p_rate,2),
                       round(net_tf_rate,2),   round(net_eq_rate,2)],
    "Corpus (₹)":     (weights * corpus_yield).round(0),
})
df["Annual Income (₹)"]  = (df["Corpus (₹)"] * df["Net Rate (%)"]/100).round(0)
df["Monthly Income (₹)"] = (df["Annual Income (₹)"] / 12).round(0)
st.dataframe(df)

# --- 8. 80C Tax‑Saving Instruments ---
st.subheader("80C Tax‑Saving Allocation")
elss_invest = st.number_input("Invest in ELSS (₹)", value=150_000, step=10_000)
st.write(f"You can claim ₹{elss_invest:,.0f} under Section 80C.")

# --- 9. Corpus Accumulation Projection ---
st.subheader("Corpus Accumulation Projection")
current_age = st.number_input("Current Age", value=35, step=1)
retire_age  = st.number_input("Retirement Age", value=60, step=1)
monthly_contrib = st.number_input("Monthly Investment (₹)", value=100_000, step=10_000)
contrib_growth  = st.slider("Annual Growth in Contribution (%)", 0.0, 15.0, 5.0)

years = list(range(retire_age - current_age + 1))
ages  = [current_age + i for i in years]

# Accumulate each asset class separately
eq_values  = [curr_eq]
fd_values  = [curr_fd]
p2p_values = [curr_p2p]
tf_values  = [curr_tf]

for yr in years[1:]:
    total_contrib = monthly_contrib * 12 * (1 + contrib_growth/100) ** (yr - 1)
    eq_values.append((eq_values[-1] + total_contrib * alloc_eq / 100) * (1 + net_eq_rate/100))
    fd_values.append((fd_values[-1] + total_contrib * alloc_fd / 100) * (1 + net_fd_rate/100))
    p2p_values.append((p2p_values[-1] + total_contrib * alloc_p2p / 100) * (1 + net_p2p_rate/100))
    tf_values.append((tf_values[-1] + total_contrib * alloc_tf / 100) * (1 + net_tf_rate/100))

corpus = np.array(eq_values) + np.array(fd_values) + np.array(p2p_values) + np.array(tf_values)

df_accum = pd.DataFrame({
    "Age": ages,
    "Equity/MF": eq_values,
    "FD": fd_values,
    "P2P": p2p_values,
    "Tax-Free Bonds": tf_values,
    "Total Corpus": corpus
})
df_accum["Total (₹ Cr)"] = df_accum["Total Corpus"] / 1e7
st.dataframe(df_accum.style.format("₹{:,.0f}"))

fig, ax = plt.subplots()
ax.stackplot(ages, eq_values, fd_values, p2p_values, tf_values, labels=["Equity", "FD", "P2P", "Tax-Free Bonds"])
ax.set_ylabel("Corpus (₹)")
ax.set_title("Projected Asset Accumulation")
ax.legend(loc="upper left")
st.pyplot(fig)

Overwriting app.py


In [1]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import date
from dateutil.relativedelta import relativedelta

# Set page configuration
st.set_page_config(
    page_title="Comprehensive Retirement Planner",
    page_icon="💰",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main {
        padding: 1rem;
    }
    .stApp {
        max-width: 1200px;
        margin: 0 auto;
    }
    .metric-card {
        background-color: #f8f9fa;
        border-radius: 0.5rem;
        padding: 1rem;
        box-shadow: 0 0.15rem 1.75rem 0 rgba(58, 59, 69, 0.15);
    }
    .big-number {
        font-size: 2.5rem;
        font-weight: bold;
        color: #1f77b4;
    }
    .subtitle {
        font-size: 1rem;
        color: #666;
    }
    .header-container {
        display: flex;
        justify-content: space-between;
        align-items: center;
        margin-bottom: 1rem;
    }
    .header-title {
        font-size: 2rem;
        font-weight: bold;
        color: #333;
    }
</style>
""", unsafe_allow_html=True)

# App title and description
st.title("Comprehensive Retirement Planner 💰")
st.markdown("""
This app helps you plan for your retirement by calculating how much you need to save,
how to allocate your investments, and providing detailed projections based on various factors.
""")

# Sidebar with input parameters
st.sidebar.header("Personal Information")

# Basic personal info
current_age = st.sidebar.slider("Current Age", 18, 65, 30)
retirement_age = st.sidebar.slider("Retirement Age", current_age + 1, 80, 60)
life_expectancy = st.sidebar.slider("Life Expectancy", retirement_age + 1, 100, 85)

# Financial info
current_annual_income = st.sidebar.number_input("Current Annual Income (₹)", min_value=0, value=1000000, step=100000, format="%d")
monthly_expenses = st.sidebar.number_input("Current Monthly Expenses (₹)", min_value=0, value=50000, step=5000, format="%d")
annual_expenses = monthly_expenses * 12

st.sidebar.header("Current Investments")
current_investments = st.sidebar.number_input("Total Current Investments (₹)", min_value=0, value=1000000, step=100000, format="%d")
current_investment_breakdown = st.sidebar.expander("Current Investment Breakdown")
with current_investment_breakdown:
    fd_percent = st.slider("Fixed Deposits (%)", 0, 100, 20)
    p2p_percent = st.slider("P2P Lending (%)", 0, 100, 10)
    bonds_percent = st.slider("Tax-Free Bonds (%)", 0, 100, 30)
    equity_percent = st.slider("Equity/Mutual Funds (%)", 0, 100, 40)

    # Ensure percentages sum to 100
    total_percent = fd_percent + p2p_percent + bonds_percent + equity_percent
    if total_percent != 100:
        st.warning(f"Your investment allocation adds up to {total_percent}%. Please adjust to reach 100%.")

st.sidebar.header("Retirement Goals")
retirement_lifestyle = st.sidebar.select_slider(
    "Desired Retirement Lifestyle",
    options=["Modest", "Comfortable", "Luxurious"],
    value="Comfortable"
)

lifestyle_factor = {"Modest": 0.7, "Comfortable": 1.0, "Luxurious": 1.3}
retirement_expense_factor = lifestyle_factor[retirement_lifestyle]

st.sidebar.header("Economic Assumptions")
inflation_rate = st.sidebar.slider("Annual Inflation Rate (%)", 2.0, 10.0, 6.0, 0.1) / 100
tax_bracket = st.sidebar.selectbox("Current Income Tax Bracket", ["5%", "10%", "15%", "20%", "25%", "30%"], index=3)
tax_rate = float(tax_bracket.strip('%')) / 100

# Investment returns based on asset class
st.sidebar.header("Expected Returns")
fd_return = st.sidebar.slider("Fixed Deposit Return (%)", 3.0, 9.0, 5.5, 0.1) / 100
p2p_return = st.sidebar.slider("P2P Lending Return (%)", 8.0, 18.0, 12.0, 0.1) / 100
bond_return = st.sidebar.slider("Tax-Free Bond Return (%)", 5.0, 9.0, 7.0, 0.1) / 100
equity_return = st.sidebar.slider("Equity/MF Return (%)", 8.0, 18.0, 12.0, 0.1) / 100

# Monthly savings
monthly_savings = st.sidebar.number_input("Current Monthly Savings (₹)", min_value=0, value=20000, step=1000, format="%d")
annual_savings = monthly_savings * 12

# Show advanced options
show_advanced = st.sidebar.checkbox("Show Advanced Options")
if show_advanced:
    st.sidebar.header("Advanced Options")
    savings_increase_rate = st.sidebar.slider("Annual Increase in Savings (%)", 0.0, 10.0, 5.0, 0.1) / 100
    retirement_tax_rate = st.sidebar.slider("Expected Retirement Tax Rate (%)", 0.0, 30.0, 10.0, 0.1) / 100
    social_security = st.sidebar.number_input("Monthly Social Security/Pension (₹)", min_value=0, value=15000, step=1000, format="%d")
    annual_social_security = social_security * 12
    include_home_equity = st.sidebar.checkbox("Include Home Equity in Retirement Planning", value=False)

    if include_home_equity:
        home_value = st.sidebar.number_input("Current Home Value (₹)", min_value=0, value=5000000, step=100000, format="%d")
        home_appreciation = st.sidebar.slider("Annual Home Appreciation (%)", 0.0, 10.0, 3.0, 0.1) / 100
        home_equity_percent = st.sidebar.slider("Home Equity to Use in Retirement (%)", 0, 100, 50) / 100
else:
    savings_increase_rate = 0.05  # 5% annual increase in savings
    retirement_tax_rate = 0.10  # 10% tax rate in retirement
    annual_social_security = 180000  # ₹15,000 monthly
    include_home_equity = False
    home_value = 5000000
    home_appreciation = 0.03
    home_equity_percent = 0.5

# Calculate retirement needs
years_to_retirement = retirement_age - current_age
retirement_years = life_expectancy - retirement_age

# Calculate expected expenses at retirement (accounting for inflation)
retirement_annual_expenses = annual_expenses * ((1 + inflation_rate) ** years_to_retirement) * retirement_expense_factor

# Calculate total retirement needs
total_retirement_need = retirement_annual_expenses * retirement_years
retirement_corpus_needed = total_retirement_need - (annual_social_security * retirement_years)

# Calculate the required monthly savings
# This is simplified and doesn't account for compounding of existing investments
current_portfolio_value_at_retirement = current_investments * (
    (1 + fd_return) ** years_to_retirement * (fd_percent / 100) +
    (1 + p2p_return) ** years_to_retirement * (p2p_percent / 100) +
    (1 + bond_return) ** years_to_retirement * (bonds_percent / 100) +
    (1 + equity_return) ** years_to_retirement * (equity_percent / 100)
)

# Calculate home equity at retirement if included
if include_home_equity:
    home_equity_at_retirement = home_value * ((1 + home_appreciation) ** years_to_retirement) * home_equity_percent
else:
    home_equity_at_retirement = 0

# Adjust need based on current portfolio and home equity
adjusted_retirement_need = retirement_corpus_needed - current_portfolio_value_at_retirement - home_equity_at_retirement

# Now calculate the required monthly savings to reach the adjusted need
# This is a simplified formula and doesn't account for varying returns across asset classes
avg_return = (fd_return + p2p_return + bond_return + equity_return) / 4
monthly_investment_needed = 0

if adjusted_retirement_need > 0:
    # Formula for future value of increasing payments (monthly savings that grow annually)
    # This is a simplification
    if avg_return == savings_increase_rate:
        # Special case when rates are equal
        monthly_investment_needed = adjusted_retirement_need / (years_to_retirement * 12) / (1 + avg_return) ** years_to_retirement
    else:
        r = (1 + avg_return) / (1 + savings_increase_rate) - 1
        monthly_investment_needed = adjusted_retirement_need * r / ((1 + r) ** years_to_retirement - 1) / 12

# Calculate optimal asset allocation based on age and risk profile
# This is a simple rule-based approach; in reality, this would be more sophisticated
# Rule: 100 - age = equity allocation
base_equity_allocation = 100 - current_age
risk_factor = {"Modest": 0.8, "Comfortable": 1.0, "Luxurious": 1.2}
adjusted_equity_allocation = min(base_equity_allocation * risk_factor[retirement_lifestyle], 90)

# Now distribute the remaining across fixed income instruments
remaining_allocation = 100 - adjusted_equity_allocation
fd_allocation = remaining_allocation * 0.3  # 30% of remaining to FDs
bond_allocation = remaining_allocation * 0.5  # 50% of remaining to bonds
p2p_allocation = remaining_allocation * 0.2  # 20% of remaining to P2P

# Round to nearest integer
adjusted_equity_allocation = round(adjusted_equity_allocation)
fd_allocation = round(fd_allocation)
bond_allocation = round(bond_allocation)
p2p_allocation = round(p2p_allocation)

# Adjust to ensure they sum to 100%
total = adjusted_equity_allocation + fd_allocation + bond_allocation + p2p_allocation
if total != 100:
    # Add or subtract the difference to/from the largest allocation
    max_allocation_name = max(
        ("Equity", adjusted_equity_allocation),
        ("FD", fd_allocation),
        ("Bond", bond_allocation),
        ("P2P", p2p_allocation),
        key=lambda x: x[1]
    )[0]

    if max_allocation_name == "Equity":
        adjusted_equity_allocation += (100 - total)
    elif max_allocation_name == "FD":
        fd_allocation += (100 - total)
    elif max_allocation_name == "Bond":
        bond_allocation += (100 - total)
    else:
        p2p_allocation += (100 - total)

# Main dashboard
st.header("Retirement Summary")

# Create columns for metrics
col1, col2, col3 = st.columns(3)

with col1:
    st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
    st.markdown(f"<div class='big-number'>₹{retirement_corpus_needed:,.0f}</div>", unsafe_allow_html=True)
    st.markdown("<div class='subtitle'>Total Retirement Corpus Needed</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)

with col2:
    st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
    st.markdown(f"<div class='big-number'>₹{monthly_investment_needed:,.0f}</div>", unsafe_allow_html=True)
    st.markdown("<div class='subtitle'>Monthly Investment Needed</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)

with col3:
    st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
    st.markdown(f"<div class='big-number'>{years_to_retirement}</div>", unsafe_allow_html=True)
    st.markdown("<div class='subtitle'>Years to Retirement</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)

# Additional metrics
col1, col2 = st.columns(2)

with col1:
    st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
    st.markdown(f"<div class='big-number'>₹{retirement_annual_expenses:,.0f}</div>", unsafe_allow_html=True)
    st.markdown("<div class='subtitle'>Annual Expenses at Retirement</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)

with col2:
    st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
    st.markdown(f"<div class='big-number'>₹{current_portfolio_value_at_retirement:,.0f}</div>", unsafe_allow_html=True)
    st.markdown("<div class='subtitle'>Current Portfolio Value at Retirement</div>", unsafe_allow_html=True)
    st.markdown("</div>", unsafe_allow_html=True)

# Gap analysis
st.header("Retirement Gap Analysis")

# Calculate monthly gap
monthly_gap = monthly_investment_needed - monthly_savings
gap_percentage = (monthly_gap / monthly_investment_needed) * 100 if monthly_investment_needed > 0 else 0

if monthly_gap > 0:
    st.warning(f"You need to increase your monthly savings by ₹{monthly_gap:,.0f} ({gap_percentage:.1f}%) to reach your retirement goal.")
else:
    st.success(f"You're on track to exceed your retirement goal by ₹{abs(monthly_gap):,.0f} per month.")

# Progress bar for retirement savings
progress = min(100, (current_portfolio_value_at_retirement / retirement_corpus_needed) * 100) if retirement_corpus_needed > 0 else 0
st.progress(progress)
st.caption(f"Current progress: {progress:.1f}% of retirement corpus needed")

# Asset allocation recommendations
st.header("Recommended Asset Allocation")

# Create data for the pie chart
recommended_allocation = {
    "Fixed Deposits": fd_allocation,
    "P2P Lending": p2p_allocation,
    "Tax-Free Bonds": bond_allocation,
    "Equity/MF": adjusted_equity_allocation
}

# Create pie chart
fig_alloc = px.pie(
    values=list(recommended_allocation.values()),
    names=list(recommended_allocation.keys()),
    title="Recommended Asset Allocation",
    color_discrete_sequence=px.colors.qualitative.Set2,
    hole=0.4
)
fig_alloc.update_traces(textposition='inside', textinfo='percent+label')
st.plotly_chart(fig_alloc, use_container_width=True)

# Detailed allocation table
col1, col2 = st.columns(2)

with col1:
    st.subheader("Current Allocation")
    current_alloc_df = pd.DataFrame({
        "Asset Class": ["Fixed Deposits", "P2P Lending", "Tax-Free Bonds", "Equity/MF"],
        "Percentage": [fd_percent, p2p_percent, bonds_percent, equity_percent],
        "Amount (₹)": [
            current_investments * fd_percent / 100,
            current_investments * p2p_percent / 100,
            current_investments * bonds_percent / 100,
            current_investments * equity_percent / 100
        ]
    })
    st.dataframe(current_alloc_df.style.format({
        "Percentage": "{:.1f}%",
        "Amount (₹)": "{:,.0f}"
    }), hide_index=True, use_container_width=True)

with col2:
    st.subheader("Recommended Allocation")
    recommended_alloc_df = pd.DataFrame({
        "Asset Class": list(recommended_allocation.keys()),
        "Percentage": list(recommended_allocation.values()),
        "Monthly Investment (₹)": [
            monthly_investment_needed * fd_allocation / 100,
            monthly_investment_needed * p2p_allocation / 100,
            monthly_investment_needed * bond_allocation / 100,
            monthly_investment_needed * adjusted_equity_allocation / 100
        ]
    })
    st.dataframe(recommended_alloc_df.style.format({
        "Percentage": "{:.1f}%",
        "Monthly Investment (₹)": "{:,.0f}"
    }), hide_index=True, use_container_width=True)

# Projection over time
st.header("Retirement Corpus Projection")

# Generate projection data
years = list(range(current_age, life_expectancy + 1))
corpus_values = []
expenses_values = []

# Initial values
corpus = current_investments
annual_expense = annual_expenses
monthly_investment = monthly_savings
retirement_reached = False

for age in years:
    # Update expenses with inflation
    annual_expense *= (1 + inflation_rate)

    # If not retired yet, add savings and investment growth
    if age < retirement_age:
        # Investment growth
        corpus *= (1 + (
            fd_return * fd_allocation / 100 +
            p2p_return * p2p_allocation / 100 +
            bond_return * bond_allocation / 100 +
            equity_return * adjusted_equity_allocation / 100
        ))

        # Add annual savings
        corpus += monthly_investment * 12

        # Increase monthly investment according to savings increase rate
        monthly_investment *= (1 + savings_increase_rate)

        expenses_values.append(0)  # No retirement expenses yet
    else:
        if not retirement_reached:
            # First year of retirement
            retirement_reached = True
            # Add home equity if included
            if include_home_equity:
                corpus += home_equity_at_retirement

        # Apply retirement tax
        effective_return = (
            fd_return * fd_allocation / 100 +
            p2p_return * p2p_allocation / 100 +
            bond_return * bond_allocation / 100 +
            equity_return * adjusted_equity_allocation / 100
        ) * (1 - retirement_tax_rate)

        # Investment growth
        corpus *= (1 + effective_return)

        # Subtract annual expenses and add social security
        corpus -= annual_expense
        corpus += annual_social_security

        expenses_values.append(annual_expense)

    corpus_values.append(max(0, corpus))

# Create projection dataframe
projection_df = pd.DataFrame({
    "Age": years,
    "Corpus Value": corpus_values,
    "Retirement Expenses": expenses_values
})

# Plot projection
fig = px.line(
    projection_df,
    x="Age",
    y=["Corpus Value", "Retirement Expenses"],
    title="Retirement Corpus Projection",
    labels={"value": "Amount (₹)", "variable": "Type"},
    color_discrete_sequence=["#1f77b4", "#ff7f0e"]
)

# Add vertical line at retirement age
fig.add_vline(x=retirement_age, line_dash="dash", line_color="red")
fig.add_annotation(
    x=retirement_age,
    y=max(corpus_values)/2,
    text=f"Retirement Age: {retirement_age}",
    showarrow=True,
    arrowhead=1
)

st.plotly_chart(fig, use_container_width=True)

# Check if corpus runs out
last_corpus = corpus_values[-1]
if last_corpus <= 0:
    # Find when corpus runs out
    for i, value in enumerate(corpus_values):
        if value <= 0:
            out_of_money_age = years[i]
            st.error(f"⚠️ WARNING: You may run out of money at age {out_of_money_age}. Consider adjusting your retirement plan.")
            break
else:
    st.success(f"✅ Your retirement plan looks sustainable with a projected corpus of ₹{last_corpus:,.0f} at age {life_expectancy}.")

# Tax implications
st.header("Tax Implications")

# Create a simple tax table
tax_table = pd.DataFrame({
    "Asset Class": ["Fixed Deposits", "P2P Lending", "Tax-Free Bonds", "Equity/MF (Long-term)"],
    "Taxability": ["Fully Taxable as Income", "Fully Taxable as Income", "Tax-free Interest", "10% above ₹1 Lakh (LTCG)"],
    "Tax Efficiency Rating": ["Low", "Low", "High", "Medium"]
})

st.dataframe(tax_table, hide_index=True, use_container_width=True)

st.markdown("""
### Tax Considerations:
- **FDs and P2P Lending**: Interest is added to your income and taxed at your income tax slab rate.
- **Tax-Free Bonds**: Interest earned is completely tax-free, making them efficient for higher tax brackets.
- **Equity/Mutual Funds**: Long-term capital gains (>1 year) are taxed at 10% above ₹1 Lakh exemption.
- **Retirement Accounts**: Consider tax-advantaged accounts like PPF, EPF, and NPS for tax benefits.
""")

# Sensitivity analysis
st.header("Sensitivity Analysis")

# Create tabs for different analyses
tab1, tab2, tab3 = st.tabs(["Return Rates", "Inflation Impact", "Retirement Age"])

with tab1:
    # Analyze impact of different return rates
    returns_range = np.linspace(avg_return - 0.03, avg_return + 0.03, 5)
    returns_labels = [f"{r*100:.1f}%" for r in returns_range]

    final_corpus_values = []

    for ret in returns_range:
        # Recalculate with this return rate
        test_corpus = current_investments
        test_monthly = monthly_savings

        for age in range(current_age, life_expectancy + 1):
            if age < retirement_age:
                test_corpus *= (1 + ret)
                test_corpus += test_monthly * 12
                test_monthly *= (1 + savings_increase_rate)
            else:
                test_corpus *= (1 + ret * (1 - retirement_tax_rate))
                test_corpus -= annual_expenses * ((1 + inflation_rate) ** (age - current_age)) * retirement_expense_factor
                test_corpus += annual_social_security

        final_corpus_values.append(max(0, test_corpus))

    # Create bar chart
    fig_returns = px.bar(
        x=returns_labels,
        y=final_corpus_values,
        title="Impact of Different Return Rates on Final Corpus",
        labels={"x": "Average Return Rate", "y": "Final Corpus (₹)"}
    )
    st.plotly_chart(fig_returns, use_container_width=True)

with tab2:
    # Analyze impact of different inflation rates
    inflation_range = np.linspace(inflation_rate - 0.02, inflation_rate + 0.02, 5)
    inflation_labels = [f"{r*100:.1f}%" for r in inflation_range]

    inflation_corpus_values = []

    for inf in inflation_range:
        # Recalculate with this inflation rate
        test_corpus = current_investments
        test_monthly = monthly_savings
        test_annual_expense = annual_expenses

        for age in range(current_age, life_expectancy + 1):
            test_annual_expense *= (1 + inf)

            if age < retirement_age:
                test_corpus *= (1 + avg_return)
                test_corpus += test_monthly * 12
                test_monthly *= (1 + savings_increase_rate)
            else:
                test_corpus *= (1 + avg_return * (1 - retirement_tax_rate))
                test_corpus -= test_annual_expense * retirement_expense_factor
                test_corpus += annual_social_security

        inflation_corpus_values.append(max(0, test_corpus))

    # Create bar chart
    fig_inflation = px.bar(
        x=inflation_labels,
        y=inflation_corpus_values,
        title="Impact of Different Inflation Rates on Final Corpus",
        labels={"x": "Inflation Rate", "y": "Final Corpus (₹)"}
    )
    st.plotly_chart(fig_inflation, use_container_width=True)

with tab3:
    # Analyze impact of different retirement ages
    retirement_range = [retirement_age - 5, retirement_age - 2, retirement_age, retirement_age + 2, retirement_age + 5]
    retirement_range = [max(current_age + 1, age) for age in retirement_range]  # Ensure retirement age > current age
    retirement_labels = [f"Age {r}" for r in retirement_range]

    retirement_corpus_values = []

    for ret_age in retirement_range:
        # Recalculate with this retirement age
        test_corpus = current_investments
        test_monthly = monthly_savings
        test_annual_expense = annual_expenses

        for age in range(current_age, life_expectancy + 1):
            test_annual_expense *= (1 + inflation_rate)

            if age < ret_age:
                test_corpus *= (1 + avg_return)
                test_corpus += test_monthly * 12
                test_monthly *= (1 + savings_increase_rate)
            else:
                test_corpus *= (1 + avg_return * (1 - retirement_tax_rate))
                test_corpus -= test_annual_expense * retirement_expense_factor
                test_corpus += annual_social_security

        retirement_corpus_values.append(max(0, test_corpus))

    # Create bar chart
    fig_retirement = px.bar(
        x=retirement_labels,
        y=retirement_corpus_values,
        title="Impact of Different Retirement Ages on Final Corpus",
        labels={"x": "Retirement Age", "y": "Final Corpus (₹)"}
    )
    st.plotly_chart(fig_retirement, use_container_width=True)

# Investment strategy recommendations
st.header("Investment Strategy Recommendations")

# Age-based strategy
years_to_retirement = retirement_age - current_age
risk_profile = ""

if years_to_retirement > 20:
    risk_profile = "Aggressive Growth"
    strategy = "Focus heavily on equity, with a small allocation to bonds for stability."
elif years_to_retirement > 10:
    risk_profile = "Growth"
    strategy = "Balanced approach with a tilt toward equity for long-term growth."
elif years_to_retirement > 5:
    risk_profile = "Moderate"
    strategy = "Equal balance between equity and fixed income to manage risk."
else:
    risk_profile = "Conservative"
    strategy = "Focus on capital preservation with higher allocation to fixed income."

st.markdown(f"""
### Based on your timeline of {years_to_retirement} years to retirement:
- **Risk Profile**: {risk_profile}
- **Strategy**: {strategy}
""")

# Specific recommendations based on inputs
st.markdown("### Specific Recommendations:")

recommendations = []

# Check if monthly gap is positive
if monthly_gap > 0:
    recommendations.append(f"Increase your monthly savings by ₹{monthly_gap:,.0f} to reach your retirement goal.")

# Check tax efficiency
if tax_rate > 0.2 and bond_allocation < 30:
    recommendations.append("Consider increasing allocation to tax-free bonds to improve tax efficiency.")

# Check equity allocation based on age
if current_age < 40 and adjusted_equity_allocation < 60:
    recommendations.append("Your age suggests you could increase equity exposure for better long-term growth.")
elif current_age > 50 and adjusted_equity_allocation > 60:
    recommendations.append("Consider reducing equity exposure to protect your portfolio as you approach retirement.")

# Retirement age recommendation
if monthly_gap > 0.3 * monthly_savings:
    recommendations.append(f"Consider delaying retirement by 2-3 years to significantly improve corpus sustainability.")

# Expense management
if annual_expenses > 0.5 * current_annual_income:
    recommendations.append("Your expenses are high relative to income. Consider expense reduction strategies.")

# Show recommendations as bullet points
for rec in recommendations:
    st.markdown(f"- {rec}")

if not recommendations:
    st.markdown("- Your retirement plan appears to be well-structured based on your inputs.")

# Investment considerations
st.markdown("""
### Additional Investment Vehicles to Consider:
- **National Pension System (NPS)**: Tax-efficient retirement vehicle with partial tax exemption
- **Public Provident Fund (PPF)**: Tax-free interest with sovereign guarantee
- **Equity-Linked Savings Scheme (ELSS)**: Tax-saving mutual funds with 3-year lock-in
- **Senior Citizen Savings Scheme (SCSS)**: High-interest government scheme for retirees
- **Real Estate Investment**: Consider REITs for real estate exposure with liquidity
""")

# Footer with disclaimers
st.markdown("---")
st.caption("""
**Disclaimer**: This retirement planner provides estimates based on the information you provide and various assumptions.
Actual results may vary due to market conditions, changes in tax laws, and other factors.
This tool is designed for educational purposes only and should not be considered financial advice.
Please consult with a qualified financial advisor before making investment decisions.
""")

# Download projection data
st.header("Download Data")
csv = projection_df.to_csv(index=False)
st.download_button(
    label="Download Projection Data",
    data=csv,
    file_name="retirement_projection.csv",
    mime="text/csv"
)

Overwriting app.py


## Run streamlit in background

In [1]:
!streamlit run /content/app.py &>/content/logs.txt &

## Expose the port 8501
Then just click in the `url` showed.

A `log.txt`file will be created.

In [ ]:
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.239.197.172
⠙⠹⠸

[![ko-fi](https://www.ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Y8Y3VYYE)